In [55]:
import geopandas as gpd
import pandas as pd
import os
import requests
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import LineString

In [70]:
q1 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q1.csv')
q2 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q2.csv')
q3 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q3.csv')
q4 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q4.csv')
census_tract = gpd.read_file('/Users/xilver/Desktop/CASA0003/Group Assessment/24137945_CASA0003_individual_visualisation/geojson/Census_Tracts_2010.geojson')


/var/folders/ng/k_cplh696b5gn6zhx9znmp100000gn/T/ipykernel_31838/347346336.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  q1 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q1.csv')
/var/folders/ng/k_cplh696b5gn6zhx9znmp100000gn/T/ipykernel_31838/347346336.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  q2 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q2.csv')
/var/folders/ng/k_cplh696b5gn6zhx9znmp100000gn/T/ipykernel_31838/347346336.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  q3 = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-trips-2024-q3.csv')
/var/folders/ng/k_cplh696b5gn6zhx9znmp100000gn/T/ipykernel_31838/347346336.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on impo

In [71]:
station = pd.read_csv('/Users/xilver/Desktop/CASA0003/Group Assessment/Data/indego-stations-2025-04-01.csv')
stations_start = station.rename(columns={
    'Station_ID': 'start_station',
    'Station_Name': 'start_station_name'
})
stations_end = station.rename(columns={
    'Station_ID': 'end_station',
    'Station_Name': 'end_station_name'
})

q1 = q1.merge(stations_start, on='start_station', how='left')
q1 = q1.merge(stations_end, on='end_station', how='left')

q2 = q2.merge(stations_start, on='start_station', how='left')
q2 = q2.merge(stations_end, on='end_station', how='left')

q3 = q3.merge(stations_start, on='start_station', how='left')
q3 = q3.merge(stations_end, on='end_station', how='left')

q4 = q4.merge(stations_start, on='start_station', how='left')
q4 = q4.merge(stations_end, on='end_station', how='left')


In [72]:
keep_columns = [
    "trip_id", "duration", "start_time", "end_time",
    "start_station_name", "start_lat", "start_lon",
    "end_station_name", "end_lat", "end_lon",
    "bike_id", "plan_duration", "trip_route_category",
    "passholder_type", "bike_type"
]
q1 = q1[keep_columns]
q2 = q2[keep_columns]
q3 = q3[keep_columns]
q4 = q4[keep_columns]

In [73]:
q1['start_time'] = pd.to_datetime(q1['start_time'])
q1['end_time'] = pd.to_datetime(q1['end_time'])

q2['start_time'] = pd.to_datetime(q2['start_time'])
q2['end_time'] = pd.to_datetime(q2['end_time'])

q3['start_time'] = pd.to_datetime(q3['start_time'])
q3['end_time'] = pd.to_datetime(q3['end_time'])

q4['start_time'] = pd.to_datetime(q4['start_time'])
q4['end_time'] = pd.to_datetime(q4['end_time'])

In [74]:
q1 = q1.dropna(subset=['start_station_name', 'end_station_name', 'start_lat', 'start_lon', 'end_lat', 'end_lon'])
q2 = q2.dropna(subset=['start_station_name', 'end_station_name', 'start_lat', 'start_lon', 'end_lat', 'end_lon'])
q3 = q3.dropna(subset=['start_station_name', 'end_station_name', 'start_lat', 'start_lon', 'end_lat', 'end_lon'])
q4 = q4.dropna(subset=['start_station_name', 'end_station_name', 'start_lat', 'start_lon', 'end_lat', 'end_lon'])

q1['od_pair'] = q1['start_station_name'].astype(str) + '-' + q1['end_station_name'].astype(str)
q2['od_pair'] = q2['start_station_name'].astype(str) + '-' + q2['end_station_name'].astype(str)
q3['od_pair'] = q3['start_station_name'].astype(str) + '-' + q3['end_station_name'].astype(str)
q4['od_pair'] = q4['start_station_name'].astype(str) + '-' + q4['end_station_name'].astype(str)


In [75]:
all_data = pd.concat([q1, q2, q3, q4], ignore_index=True)
all_data = all_data[all_data['trip_route_category'] != 'Round Trip']
all_data = all_data[all_data['start_station_name'] != all_data['end_station_name']]
all_data['month'] = all_data['start_time'].dt.month

for month in range(1, 13):
    df_month = all_data[all_data['month'] == month]
    
    od_matrix = df_month.groupby('od_pair').agg(
    start_station_name=('start_station_name', 'first'),
    end_station_name=('end_station_name', 'first'),
    start_lat=('start_lat', 'first'),
    start_lon=('start_lon', 'first'),
    end_lat=('end_lat', 'first'),
    end_lon=('end_lon', 'first'),
    month=('month', 'first'))

    od_matrix['trip_count'] = df_month.groupby('od_pair').size().values
    od_matrix['geometry'] = od_matrix.apply(lambda row: LineString([(row['start_lon'], row['start_lat']), (row['end_lon'], row['end_lat'])]), axis=1)
    od_gdf = gpd.GeoDataFrame(od_matrix, geometry='geometry', crs='EPSG:4326')
    output_path = f'/Users/xilver/Desktop/CASA0003/Group Assessment/Monthly Flow/od_flows_month_{month}.geojson'
    od_gdf.to_file(output_path, driver='GeoJSON')

od_matrix

In [76]:
print("Monthly Trips：")
for month in range(1, 13):
    df_month = all_data[all_data['month'] == month]
    print(f"Month {month}: {len(df_month)} records")

Monthly Trips：
Month 1: 46577 records
Month 2: 59688 records
Month 3: 73944 records
Month 4: 94302 records
Month 5: 115933 records
Month 6: 127584 records
Month 7: 125188 records
Month 8: 122868 records
Month 9: 127341 records
Month 10: 128789 records
Month 11: 92801 records
Month 12: 56056 records


In [78]:
all_data['start_date'] = all_data['start_time'].dt.date
all_data['end_date'] = all_data['end_time'].dt.date
borrow_daily = all_data.groupby(['start_station_name', 'month', 'start_date']).size().reset_index(name='borrow_count')

return_daily = all_data.groupby(['end_station_name', 'month', 'end_date']).size().reset_index(name='return_count')

stations = all_data[['start_station_name', 'start_lat', 'start_lon']].drop_duplicates().rename(columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lon': 'lon'})
borrow_daily = borrow_daily.merge(stations, left_on='start_station_name', right_on='station_name', how='left')
return_daily = return_daily.merge(stations, left_on='end_station_name', right_on='station_name', how='left')

daily_activity = pd.merge(borrow_daily, return_daily, 
                         left_on=['station_name', 'month', 'start_date'], 
                         right_on=['station_name', 'month', 'end_date'], 
                         how='outer', 
                         suffixes=('_borrow', '_return')).fillna(0)


daily_activity['net_activity'] = daily_activity['borrow_count'] - daily_activity['return_count']
daily_activity = daily_activity.rename(columns={'start_date': 'date'}).drop(columns=['end_date'])

def fill_dates(df, stations):
    stations_list = stations['station_name'].unique()
   
    dates = []
    for month in range(1, 13):
        month_dates = pd.date_range(start=f'2024-{month:02d}-01', end=f'2024-{month:02d}-{pd.Period(f'2024-{month:02d}').days_in_month}', freq='D').date
        for date in month_dates:
            for station in stations_list:
                dates.append({'station_name': station, 'month': month, 'date': date})
    full_df = pd.DataFrame(dates)

    daily_activity_filled = pd.merge(full_df, df, on=['station_name', 'month', 'date'], how='left').fillna(0)

    daily_activity_filled = daily_activity_filled.merge(stations, on='station_name', how='left')
    daily_activity_filled['net_activity'] = daily_activity_filled['borrow_count'] - daily_activity_filled['return_count']
    
    return daily_activity_filled

daily_activity = fill_dates(daily_activity, stations)
columns_to_keep = ['station_name', 'month', 'date', 'borrow_count', 'return_count', 'net_activity', 'lat', 'lon']
daily_activity = daily_activity[columns_to_keep]
daily_activity_sorted = daily_activity.sort_values(by=['station_name', 'date'])

In [79]:
daily_gdf = gpd.GeoDataFrame(daily_activity_sorted, 
                             geometry=gpd.points_from_xy(daily_activity_sorted['lon'], daily_activity_sorted['lat']), 
                             crs='EPSG:4326')

for month in range(1, 13):
    month_gdf = daily_gdf[daily_gdf['month'] == month]
    if not month_gdf.empty:
        output_path = f'/Users/xilver/Desktop/CASA0003/Group Assessment/Net Count/station_daily_activity_month_{month}.geojson'
        month_gdf.to_file(output_path, driver='GeoJSON')



In [50]:
daily_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106202 entries, 0 to 106201
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   station_name        106202 non-null  object 
 1   month               106202 non-null  int64  
 2   date                106202 non-null  object 
 3   start_station_name  106202 non-null  object 
 4   borrow_count        106202 non-null  float64
 5   lat_borrow          106202 non-null  float64
 6   lon_borrow          106202 non-null  float64
 7   end_station_name    106202 non-null  object 
 8   return_count        106202 non-null  float64
 9   lat_return          106202 non-null  float64
 10  lon_return          106202 non-null  float64
 11  net_activity        106202 non-null  float64
 12  lat                 106202 non-null  float64
 13  lon                 106202 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 11.3+ MB


In [ ]:
all_data